In [1]:
from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
import numpy as np
import pandas as pd
#import seaborn as sns

https://www.imdb.com/search/title/?title_type=feature&num_votes=500,&genres=horror,mystery,!documentary,!short&sort=user_rating,desc


https://www.imdb.com/search/title/?title_type=feature,tv_series&num_votes=1000,&genres=horror,mystery,!documentary,!short&sort=user_rating,desc

# Get data

In [33]:
# load datasets
title_basics = pd.read_csv('title.basics.tsv',sep='\t')
title_ratings = pd.read_csv('title.ratings.tsv',sep='\t')
title_akas = pd.read_csv('title.akas.tsv',sep='\t')

C:\Users\lisa.hornung\AppData\Local\Temp\ipykernel_24044\3466784569.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_basics = pd.read_csv('title.basics.tsv',sep='\t')


In [91]:
# clean start Year
startYear_clean=[]

for i in range(len(title_basics["startYear"])):
    if len(title_basics["startYear"][i])>2:
       startYear_clean.append(int(title_basics["startYear"][i]))     
    else:
        startYear_clean.append(None)     

title_basics["startYear_clean"] = startYear_clean

In [103]:
# find horror and mystery films
title_basics.loc[title_basics["genres"].str.find("Horror")!=-1, "Horror"] = 1
title_basics.loc[title_basics["genres"].str.find("Mystery")!=-1, "Mystery"] = 1


# exclude documentaries and talk/news shows
title_basics.loc[title_basics["genres"].str.find("Documentary")!=-1, "Documentary"] = 1
title_basics.loc[title_basics["genres"].str.find("Talk-Show")!=-1, "Talk-Show"] = 1
title_basics.loc[title_basics["genres"].str.find("News")!=-1, "News"] = 1

# create dummy for comedy
title_basics.loc[title_basics["genres"].str.find("Comedy")!=-1, "Comedy"] = 1

In [112]:
# get filtered list of horror films since 1960
horror_films = title_basics[((title_basics["Mystery"]==1) | (title_basics["Horror"]==1))
              & (title_basics["Documentary"]!=1) & (title_basics["Talk-Show"]!=1) & (title_basics["News"]!=1)
              & (title_basics["startYear_clean"]>=1960)
              & (title_basics["titleType"].isin(["movie", "tvSeries"]))].sort_values(by="startYear_clean").reset_index(drop=True)
#.to_csv("horror_films_1960_2024.csv", index=False)

In [118]:
# merge ratings
horror_films = pd.merge(horror_films, title_ratings, left_on="tconst", right_on="tconst", how="left")

In [119]:
horror_films.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,Horror,Mystery,startYear_clean,Documentary,Talk-Show,News,Comedy,averageRating,numVotes
0,tt0053490,tvSeries,The Case of the Dangerous Robin,The Case of the Dangerous Robin,0,1960,\N,30,"Crime,Drama,Mystery",NaN,1.0,1960.0,NaN,NaN,NaN,NaN,8.1,57.0
1,tt1190177,movie,Mga alamat ng sandaigdig,Mga alamat ng sandaigdig,0,1960,\N,90,Horror,1.0,NaN,1960.0,NaN,NaN,NaN,NaN,5.9,45.0
2,tt12042772,movie,The Secret,Himitsu,0,1960,\N,97,Mystery,NaN,1.0,1960.0,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0051874,movie,La Llorona,La Llorona,0,1960,\N,75,"Drama,Fantasy,Horror",1.0,NaN,1960.0,NaN,NaN,NaN,NaN,5.8,150.0
4,tt0055226,movie,"Neutrón, el enmascarado negro","Neutrón, el enmascarado negro",0,1960,\N,72,"Action,Drama,Mystery",NaN,1.0,1960.0,NaN,NaN,NaN,NaN,5.3,68.0


In [121]:
# export
horror_films.to_csv("horror_films_1960_2024.csv", index=False)

## Webscraping

Useful guide: [webscraping sci-fi movies from imdb](https://www.freecodecamp.org/news/web-scraping-sci-fi-movies-from-imdb-with-python/)

--> not possible anymore, seems webscraping is now blocked by IMDB. But you can get their non-commercial data sets [here](https://datasets.imdbws.com/)

In [13]:
# Note this takes about 40 min to run if np.arange is set to 9951 as the stopping point.

pages = np.arange(1, 2001, 50) # Last time I tried, I could only go to 10000 items because after that the URI has no discernable pattern to combat webcrawlers; I just did 4 pages for demonstration purposes. You can increase this for your own projects.
#headers = {'Accept-Language': 'en-US,en;q=0.8'} # If this is not specified, the default language is Mandarin

#initialize empty lists to store the variables scraped
titles = []
years = []
ratings = []
genres = []
runtimes = []
imdb_ratings = []
imdb_ratings_standardized = []
metascores = []
votes = []

for page in pages:
    print(page)

    #get request for sci-fi
    response = get("https://www.imdb.com/search/title?title_type=feature,tv_series&genres=horror,mystery,!documentary,!short&num_votes=1000&"
    #"https://www.imdb.com/search/title?genres=sci-fi&"
    + "start="
    + str(page)
    + "&explore=title_type,genres&ref_=adv_prv", headers=headers)
  
    sleep(randint(8,15))
  
    #throw warning for status codes that are not 200
    if response.status_code != 200:
        #warn('Request: {}; Status code: {}'.format(requests, response.status_code))
        warn('Request: {}; Status code: {}'.format(response, response.status_code))

    #parse the content of current iteration of request
    page_html = BeautifulSoup(response.text, 'html.parser')
    movie_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
  
    #extract the 50 movies for that page
    for container in movie_containers:

    #conditional for all with metascore
        if container.find('div', class_ = 'ratings-metascore') is not None:

            #title
            title = container.h3.a.text
            titles.append(title)
        else:
            titles.append(None)

        if container.h3.find('span', class_= 'lister-item-year text-muted unbold') is not None:
            
            #year released
            year = container.h3.find('span', class_= 'lister-item-year text-muted unbold').text # remove the parentheses around the year and make it an integer
            years.append(year)

        else:
            years.append(None) # each of the additional if clauses are to handle type None data, replacing it with an empty string so the arrays are of the same length at the end of the scraping
        
        #rating
        if container.p.find('span', class_ = 'certificate') is not None:
            rating = container.p.find('span', class_= 'certificate').text
            ratings.append(rating)

        else:
            ratings.append("")

        #genre
        if container.p.find('span', class_ = 'genre') is not None:
            genre = container.p.find('span', class_ = 'genre').text.replace("\n", "").rstrip().split(',') # remove the whitespace character, strip, and split to create an array of genres
            genres.append(genre)
          
        else:
            genres.append("")

        #runtime
        if container.p.find('span', class_ = 'runtime') is not None:
            time = int(container.p.find('span', class_ = 'runtime').text.replace(" min", "")) # remove the minute word from the runtime and make it an integer
            runtimes.append(time)

        else:
            runtimes.append(None)

        #IMDB ratings
        if float(container.strong.text) is not None:
            imdb = float(container.strong.text) # non-standardized variable
            imdb_ratings.append(imdb)

        else:
            imdb_ratings.append(None)

        #Metascore
        if container.find('span', class_ = 'metascore').text is not None:
            m_score = int(container.find('span', class_ = 'metascore').text) # make it an integer
            metascores.append(m_score)

        else:
            metascores.append(None)

        #Number of votes
        if container.find('span', attrs = {'name':'nv'})['data-value'] is not None:
            vote = int(container.find('span', attrs = {'name':'nv'})['data-value'])
            votes.append(vote)

        else:
            votes.append(None)

1


C:\Users\lisa.hornung\AppData\Local\Temp\ipykernel_24044\1726316424.py:32: UserWarning: Request: <Response [403]>; Status code: 403
  warn('Request: {}; Status code: {}'.format(response, response.status_code))


51
101
151
201
251
301
351
401
451
501


KeyboardInterrupt: 